This file uses a k-mer approach to create a baseline for eDNA classification.

This file was taken from Rose Gurung (rose.gurung@maine.edu). No major changes have been made other than saving the numpy arrays and the model such that they can be loaded in and time can be saved by eliminating the need to recreate them.

In [2]:
import sys
import numpy as np
import pandas as pd
from itertools import product
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from joblib import dump, load
from sklearn.preprocessing import LabelEncoder

In [143]:
np.set_printoptions(threshold=sys.maxsize)

In [3]:
# load the train data
train_df = pd.read_csv('./datasets/train.csv')
train_df.head()

,Unnamed: 0,seq,species,genus,family,order,order_cat,family_cat,genus_cat,species_cat
0,396,cccccactatagttaaacgtctattatttttatacacctaatattc...,Crenicichla saxatilis,Crenicichla,Cichlidae,Perciformes,12,16,48,44
1,389,cccccccaaataaacatctattacttttatacacctaaattcccat...,Crenicichla saxatilis,Crenicichla,Cichlidae,Perciformes,12,16,48,44
2,391,ccccccactaaataaacatctattacttttatacacctaaattccc...,Crenicichla saxatilis,Crenicichla,Cichlidae,Perciformes,12,16,48,44
3,395,ccccccccactaaataaacatctatacttttatacacctaaattcc...,Crenicichla saxatilis,Crenicichla,Cichlidae,Perciformes,12,16,48,44
4,393,cccccacataattaatacatctattacttttatacacctaaattcc...,Crenicichla saxatilis,Crenicichla,Cichlidae,Perciformes,12,16,48,44


In [145]:
train_df.shape

(936, 10)

In [146]:
# load the test data
test_df = pd.read_csv('./datasets/test.csv')
test_df.head()

,Unnamed: 0,seq,species,genus,family,order,order_cat,family_cat,genus_cat,species_cat
0,390,cccccccccccccaaataaacatctattacttttatacacctaaat...,Crenicichla saxatilis,Crenicichla,Cichlidae,Perciformes,12,16,48,44
1,394,cccccactatagttaaacgtctatatttttatacacctaatattcc...,Crenicichla saxatilis,Crenicichla,Cichlidae,Perciformes,12,16,48,44
2,384,ccccccccactataataaacatctataattttatacacctaaattc...,Crenicichla albopunctata,Crenicichla,Cichlidae,Perciformes,12,16,48,41
3,431,ccccccccactataataaacatctattaattttatacacctaaact...,Crenicichla albopunctata,Crenicichla,Cichlidae,Perciformes,12,16,48,41
4,348,cccctctcacaaccacaaaagctatataacacaacctcttacacca...,Eigenmannia virescens,Eigenmannia,Sternopygidae,Gymnotiformes,7,50,60,52


In [147]:
test_df.shape

(175, 10)

In [148]:
# get only the species and labels columns

X_train =  train_df.loc[:,['seq']].values
y_train =  train_df.loc[:,['species']].values
X_test =  test_df.loc[:,['seq']].values
y_test =  test_df.loc[:,['species']].values

In [149]:
# remove inner lists

print("Before")
print(X_train[:5])
print(X_test[:5])
print(y_train[:5])
print(y_test[:5])

X_train = X_train.ravel()
X_test = X_test.ravel()
y_train = y_train.ravel()
y_test = y_test.ravel()

print("\nAfter")
print(X_train[:5])
print(X_test[:5])
print(y_train[:5])
print(y_test[:5])


Before
[['cccccactatagttaaacgtctattatttttatacacctaatattccccggggggagacaagtcgtaa']
 ['cccccccaaataaacatctattacttttatacacctaaattcccatgggggagacaagtcgtaa']
 ['ccccccactaaataaacatctattacttttatacacctaaattcccatgggggagacaagtcgtaa']
 ['ccccccccactaaataaacatctatacttttatacacctaaattcccatggggagacaagtcgtaa']
 ['cccccacataattaatacatctattacttttatacacctaaattcccaggggggagacaagtcgtaa']]
[['cccccccccccccaaataaacatctattacttttatacacctaaattcccatgggggagacaagtcgtaa']
 ['cccccactatagttaaacgtctatatttttatacacctaatattccccggggggagacaagtcgtaa']
 ['ccccccccactataataaacatctataattttatacacctaaattcccatgggggagacaagtcgtaa']
 ['ccccccccactataataaacatctattaattttatacacctaaactcccatgggggagacaagtcgtaa']
 ['cccctctcacaaccacaaaagctatataacacaacctcttacaccaaggggaggcaagtcgtaa']]
[['Crenicichla saxatilis']
 ['Crenicichla saxatilis']
 ['Crenicichla saxatilis']
 ['Crenicichla saxatilis']
 ['Crenicichla saxatilis']]
[['Crenicichla saxatilis']
 ['Crenicichla saxatilis']
 ['Crenicichla albopunctata']
 ['Crenicichla albopunctata']
 ['Eigenmann

In [150]:
# change labels into indices

# uniq_species=np.unique(y_train)
# classes= np.unique(y_train, return_inverse=True)[1]
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [151]:
print(y_train[:5])
print(y_test[:5])

[44 44 44 44 44]
[44 44 41 41 52]


In [152]:
def get_all_kmers(k):
    """
    generates all possible combinations of the letters a, g, t, and c of length k
    Examples:
        input: get_all_kmers(1)
        output: ['a', 'g', 't', 'c'] 
        input: get_all_kmers(2)
        output: ['aa', 'ag', 'at', 'ac', 'ga', 'gg', ... ]
    """
    all_kmers=[]
    for i in product('agtc', repeat = k):
        all_kmers.append(''.join(i))
    return all_kmers

In [153]:
def list_kmers(seq, k):
    """
    generates all kmers for the sequence (cap at 200 length sequence)
    Example
        input: list_kmers('agtcagtc', 2)
        output: ['ag', 'gt', 'tc', 'ca', 'ag', 'gt', 'tc']
    """
    kmers = []
    # Calculate how many kmers of length k there are
    if len(seq)<200:
        num_kmers=len(seq)-k+1
    else:
        #only for the first 200 letters of the sequence
        num_kmers = len(seq[:200]) - k + 1 
    # Loop over the kmer start positions
    for i in range(num_kmers):
        # Slice the string to get the kmer
        kmer = seq[i:i+k]
        kmers.append(kmer)
    return kmers

In [154]:
def create_feature_table(sequences,k):
    """
    creates a 2D array (or feature table) where each row corresponds to a sequence
    from the input list sequences, and each column corresponds to a possible k-mer
    of length k. The value at a specific row and column is the count of the
    corresponding k-mer in the corresponding sequence.
    """
    feature_table=[]
    for seq in sequences:
        cv = CountVectorizer(vocabulary=(get_all_kmers(k))) #lists counts of all words
        #representing sequences as sentences with words that are kmers i.e all kmers separated by space
        features=np.asarray(cv.fit_transform([(" ".join(list_kmers(seq,k)))]).toarray())  
        features=features.flatten().tolist()
        feature_table.append(features)
    return np.asarray(feature_table)

In [155]:
# 3mer feature table

ft_3=create_feature_table(X_train, 3)
print(ft_3.shape) # (num_sequences, num_possible_kmers)
ft_3[:5]

(936, 64)


array([[1, 1, 1, 1, 1, 0, 2, 0, 3, 0, 3, 0, 2, 1, 1, 1, 0, 1, 0, 1, 1, 4,
        0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 3, 1, 5, 1, 0, 0, 0, 0, 3, 0, 3, 1,
        0, 1, 1, 1, 1, 0, 0, 2, 0, 1, 2, 0, 3, 0, 0, 0, 1, 1, 1, 5],
       [3, 1, 2, 1, 1, 0, 1, 0, 2, 1, 2, 1, 3, 0, 1, 1, 0, 1, 0, 1, 1, 3,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 3, 0, 2, 2, 0, 1, 0, 0, 2, 0, 2, 1,
        0, 1, 1, 1, 2, 0, 2, 1, 0, 0, 1, 0, 2, 0, 1, 0, 2, 0, 1, 6],
       [3, 1, 2, 1, 1, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 1, 0, 1, 0, 1, 1, 3,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 4, 0, 2, 2, 0, 1, 0, 0, 2, 0, 2, 1,
        0, 1, 1, 1, 1, 0, 2, 2, 0, 0, 1, 0, 3, 0, 1, 0, 2, 0, 1, 5],
       [3, 1, 2, 1, 1, 0, 1, 0, 3, 1, 1, 1, 3, 0, 2, 1, 0, 1, 0, 1, 1, 2,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 4, 0, 2, 2, 0, 1, 0, 0, 1, 0, 2, 1,
        0, 1, 1, 1, 1, 0, 2, 2, 0, 0, 1, 0, 3, 0, 1, 0, 2, 0, 1, 7],
       [1, 1, 3, 0, 1, 1, 1, 0, 3, 0, 3, 1, 4, 0, 1, 1, 0, 1, 0, 1, 1, 4,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 4, 0, 2, 3, 0, 0, 0,

In [156]:
# 5mer feature table

ft_5=create_feature_table(X_train, 5)
print(ft_5.shape)
ft_5[:2]

(936, 1024)


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [134]:
# 8mer feature table

ft_8=create_feature_table(X_train, 8)
len(ft_8[0])
# takes ~1 min. 65536

65536

In [ ]:
# 10mer feature table

ft_10=create_feature_table(X_train, 10)
len(ft_10[0])
# takes ~17 mins. 1048576

In [ ]:
np.save('./datasets/ft_3.npy', ft_3)
np.save('./datasets/ft_5.npy', ft_5)
np.save('./datasets/ft_8.npy', ft_8)
np.save('./datasets/ft_10.npy', ft_10)

print(f"Ft_3: {ft_3.shape}")
print(f"Ft_5: {ft_5.shape}")
print(f"Ft_8: {ft_8.shape}")
print(f"Ft_10: {ft_10.shape}")

Ft_3: 936
Ft_5: 936
Ft_8: 936
Ft_10: 936


## Resume here if you already created the train & test kmer data

In [136]:
import sys
import numpy as np
import pandas as pd
from itertools import product
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from joblib import dump, load

In [40]:
# takes ~ 45 seconds
ft_3 = np.load('./datasets/ft_3.npy')
ft_5 = np.load('./datasets/ft_5.npy')
ft_8 = np.load('./datasets/ft_8.npy')
ft_10 = np.load('./datasets/ft_10.npy')
print(f"Ft_3: {ft_3.shape}")
print(f"Ft_5: {ft_5.shape}")
print(f"Ft_8: {ft_8.shape}")
print(f"Ft_10: {ft_10.shape}")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

Ft_3: (936, 64)
Ft_5: (936, 1024)
Ft_8: (936, 65536)
Ft_10: (936, 1048576)
X_train: (936,)
y_train: (936, 1)
X_test: (175, 1)
y_test: (175, 1)


In [157]:
ft_3[:3]

array([[1, 1, 1, 1, 1, 0, 2, 0, 3, 0, 3, 0, 2, 1, 1, 1, 0, 1, 0, 1, 1, 4,
        0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 3, 1, 5, 1, 0, 0, 0, 0, 3, 0, 3, 1,
        0, 1, 1, 1, 1, 0, 0, 2, 0, 1, 2, 0, 3, 0, 0, 0, 1, 1, 1, 5],
       [3, 1, 2, 1, 1, 0, 1, 0, 2, 1, 2, 1, 3, 0, 1, 1, 0, 1, 0, 1, 1, 3,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 3, 0, 2, 2, 0, 1, 0, 0, 2, 0, 2, 1,
        0, 1, 1, 1, 2, 0, 2, 1, 0, 0, 1, 0, 2, 0, 1, 0, 2, 0, 1, 6],
       [3, 1, 2, 1, 1, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 1, 0, 1, 0, 1, 1, 3,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 4, 0, 2, 2, 0, 1, 0, 0, 2, 0, 2, 1,
        0, 1, 1, 1, 1, 0, 2, 2, 0, 0, 1, 0, 3, 0, 1, 0, 2, 0, 1, 5]])

In [158]:
y_train[:10]

array([44, 44, 44, 44, 44, 44, 41, 41, 41, 41])

In [159]:
# multinomial naive bayes
NB_3 = MultinomialNB()
NB_3.fit(ft_3, y_train)
dump(NB_3, './datasets/nb3.joblib')

['nb3.joblib']

In [160]:
NB_3 = load('./datasets/nb3.joblib')

In [161]:
# predictions with test data

ft_test=create_feature_table(X_test,3)
predicted=NB_3.predict(ft_test)

In [162]:
# accuracy with test data
# 0.00641025641 is random 1/156

NB_3.score(ft_test, y_test)

0.9371428571428572

In [163]:
predicted[:5]

array([45, 44, 41, 41, 52])

In [164]:
y_test[:5]

array([44, 44, 41, 41, 52])

In [165]:
#correctly predicted species and number of correct predictions
correct_species=[]
correct=0
for i in range(len(y_test)):
    if predicted[i]==y_test[i]:
        correct_species.append((y_test[i]))
        correct+=1

In [166]:
correct

164

In [167]:
len(np.unique(np.asarray(correct_species)))

147

In [168]:
np.unique(np.asarray(correct_species))

array([  0,   1,   2,   3,   4,   5,   6,   7,   9,  10,  11,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,
        69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
        82,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 101, 102, 103, 104, 105, 106, 107, 109, 110,
       111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155])

In [170]:
# accuracy with train data
NB_3.score(ft_3, y_train)

0.9903846153846154

In [ ]:
# def count_kmers(read, k):
#     # Start with an empty dictionary
#     counts = {}
#     # Calculate how many kmers of length k there are
#     num_kmers = len(read) - k + 1
#     # Loop over the kmer start positions
#     for i in range(num_kmers):
#         # Slice the string to get the kmer
#         kmer = read[i:i+k]
#         # Add the kmer to the dictionary if it's not there
#         if kmer not in counts:
#             counts[kmer] = 0
#         # Increment the count for this kmer
#         counts[kmer] += 1
#     # Return the final counts
#     return counts